In [1]:
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import plotly.express as px
import plotly.graph_objects as go

import datetime

from sklearn.preprocessing import MinMaxScaler
from numpy.fft import fft
# from statsmodels.graphics.tsaplots import plot_acf
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_percentage_error as mape

from holt_winters import HoltWinters

In [2]:
pjme = pd.read_csv('PJME_hourly.csv',
                   index_col=[0], parse_dates=[0]).sort_index().rename(columns={"PJME_MW": "mw"})
# pjme.head(5)

In [3]:
pjme['id'] = "pjme"
pjme['date'] = pjme.index
pjme = pjme[300:700]
y = pd.Series(MinMaxScaler().fit_transform(
    pd.DataFrame(pjme[["mw"]]))[:, 0], index=pjme.index)
df = pd.DataFrame({"x": list(range(len(y))), "y": y})

In [4]:
px.line(x=df.x, y=df.y)

### Входные параметры программы

In [5]:
from dataclasses import dataclass


@dataclass
class InputParams:
    alpha1 = 0.5
    alpha2 = 0.5
    alpha3 = 0.5
    period_length_seasons = 24  # длина периода в сезонах
    prediction_horizon = 2

In [6]:
input_params = InputParams()

### Поиск a(0), b(0)

In [7]:
lin_reg = LinearRegression()

lin_reg = lin_reg.fit(df[["x"]], df[["y"]])

In [8]:
a0, b0 = lin_reg.coef_[0][0], lin_reg.intercept_[0]
a0, b0

(-0.0005953813862412849, 0.693768106014528)

In [9]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=df.x,
        y=df.y, name="original time series")
)
fig.add_trace(
    go.Scatter(
        x=df.x,
        y=lin_reg.predict(df[["x"]])[:, 0], name="Fitted linear regression")
)
fig.update_layout(title="Поиск значений a(0), b(0) для инициализации")

In [10]:
hw = HoltWinters(df.y.to_list(),
                 slen=input_params.period_length_seasons,
                 alpha=input_params.alpha1,
                 beta=input_params.alpha2,
                 gamma=input_params.alpha3,
                 n_preds=input_params.period_length_seasons*input_params.prediction_horizon)

In [11]:
pred = hw.triple_exponential_smoothing(a0,b0)
df_pred = pd.DataFrame({"x": list(range(len(pred))),
                        "y": pred})
len(df_pred)

a0 =-0.0005953813862412849
b0 =0.693768106014528


448

In [12]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=df.x,
        y=df.y, name="original time series"))
fig.add_trace(
    go.Scatter(
        x=df_pred.x,
        y=df_pred.y, name="Predicted time series"))

fig.update_layout(title="Предсказание модели")
